---
# **1. Business Understanding**

### Business Insights (COME BACK TO THIS)
- Dublin has the highest number of bakeries listed on Yelp, indicating strong competition and demand.
- Rating distribution suggests that most bakeries in Ireland receive positive reviews.
---

# **2. Data Mining Summary**

### **Data Mining Methodology & Source Documentation**

The foundation of this project is a custom-built dataset created through systematic web scraping of Yelp.ie, fully documented in the `DataMining.ipynb` notebook. This approach ensured data authenticity and relevance to modern Irish bakery market conditions.

#### **Technical Implementation**
The data collection employed sophisticated web scraping techniques:
- **Automated Navigation**: Selenium WebDriver managed dynamic page loading and regional pagination
- **Structured Parsing**: BeautifulSoup extracted consistent business attributes from complex HTML
- **Geographical Sampling**: 12 Irish regions provided comprehensive market coverage
- **Quality Assurance**: Deduplication and validation ensured data integrity

#### **Data Collection Scope**
The scraping process captured multiple business dimensions:
- **Identity Metrics**: Business names and regional locations
- **Performance Indicators**: Star ratings and review volumes
- **Market Positioning**: Price ranges and category classifications
- **Customer Intelligence**: Review snippets and location context

#### **Technical Challenges Overcome**
The process successfully navigated several modern web challenges:
- **Dynamic Content**: JavaScript-rendered elements required strategic loading delays
- **Anti-Bot Protections**: Rate limiting and CAPTCHAs necessitated careful timing
- **Structural Volatility**: Changing CSS classes demanded robust selection strategies
- **Data Consistency**: Natural missingness patterns reflected authentic business variations

#### **Dataset Characteristics**
- **Record Count**: 1,519 initial listings, 1,488 after deduplication
- **Timeframe**: Current market snapshot (November 2024)
- **Coverage**: Urban and regional Irish bakery market
- **Completeness**: Authentic missingness patterns preserved for analytical integrity

#### **Methodological Value**
This custom data collection approach provides significant advantages over pre-existing datasets:
- **Current Relevance**: Real-time market conditions
- **Irish Specificity**: Local market focus
- **Business Intelligence**: Actionable industry insights
- **Academic Rigor**: Transparent, reproducible methodology

The `DataMining.ipynb` notebook contains complete technical documentation, including code implementations, error handling strategies, and methodological decisions that shaped the final dataset structure and quality.

---
# **3. Data Cleaning Documentation**


In this step, we classify all variables by type and purpose, handle missing values,
convert raw scraped text fields into numeric form, and prepare the dataset for
exploratory analysis and modelling.

Since the dataset comes solely from Yelp.ie, missing values occur because some
businesses do not list a price range, have no reviews yet, or lack a visible
preview snippet. These are expected and not scraping errors.

The cleaning process includes:
- assigning variable types (numerical or categorical)
- assigning variable purpose (response / explanatory)
- converting rating and review counts to numeric variables
- encoding price range (€ / €€ / €€€) as an ordinal variable
- creating additional useful features for modelling
- removing unusable rows (e.g., missing ratings for regression)

## 3.1 Load Dataset

In [42]:
import pandas as pd
import numpy as np

# Load the dataset from web scraping step
df_raw = pd.read_csv("../data/dataProject.csv")

print("Dataset shape:", df_raw.shape)
print("\nFirst few records:")
df_raw.head(3)

print("Missing values in each column:")
df_raw.isna().sum()

Dataset shape: (1488, 9)

First few records:
Missing values in each column:


source                0
region                0
name                  0
rating_raw          440
review_count_raw    516
location              0
price_range         940
categories            0
snippet             518
dtype: int64

## 3.2 Variable Classification
| Variable         | Type        | Role                          |
|:-----------------|:------------|:------------------------------|
| source           | Categorical | Metadata (ignore)             |
| region           | Categorical | Explanatory (used to predict) |
| name             | Text        | Identifier (ignore)           |
| rating_raw       | Numerical   | Response (want to predict)    |
| review_count_raw | Numerical   | Explanatory (used to predict) |
| location         | Text        | Identifier (ignore)           |
| price_range      | Categorical | Explanatory (used to predict) |
| categories       | Categorical | Explanatory (used to predict) |
| snippet          | Text        | Identifier (ignore)           |

In [43]:
print("Data types:")
print(df_raw.dtypes)

Data types:
source              object
region              object
name                object
rating_raw          object
review_count_raw    object
location            object
price_range         object
categories          object
snippet             object
dtype: object


## 3.3 Data Quality Issues Check


In [44]:
# Check for data capture errors found by scanning CSV manually
print("Checking for hours captured as ratings...")
hours_as_ratings = df_raw['rating_raw'].astype(str).str.contains('until|:', na=False).sum()
print(f"Found {hours_as_ratings} entries with opening hours instead of ratings")

if hours_as_ratings > 0:
    examples = df_raw[df_raw['rating_raw'].astype(str).str.contains('until|:', na=False)].head(3)
    for idx, row in examples.iterrows():
        print(f"  {row['name']}: '{row['rating_raw']}'")


print("Checking price range formats...")
price_values = df_raw['price_range'].dropna().unique()
print("Price formats found:", list(price_values))

# Check for mixed currencies
mixed_currencies = df_raw['price_range'].astype(str).str.contains(r'[\$£]', na=False).sum()
print(f"Entries with non-Euro symbols: {mixed_currencies}")

Checking for hours captured as ratings...
Found 78 entries with opening hours instead of ratings
  Revolution Bakery: 'until 17:00'
  GF Bakery: 'until 18:00'
  Claudia's Bakery: 'until 21:00'
Checking price range formats...
Price formats found: ['€€', '€', '€€€', '€€€€', '££', '£', '$', '$$', '$$$', '£££']
Entries with non-Euro symbols: 272


## 3.4 Clean Rating Data

In [45]:
# Create working copy
df_clean = df_raw.copy()

# Remove entries with hours instead of ratings
hours_mask = df_clean['rating_raw'].astype(str).str.contains('until|:', na=False)
df_clean = df_clean[~hours_mask]

# Convert to numeric
df_clean['rating_final'] = pd.to_numeric(df_clean['rating_raw'], errors='coerce')

# Drop remaining missing ratings for modeling integrity
df_clean = df_clean.dropna(subset=['rating_final'])

print(f"Final ratings: {len(df_clean)} records")
print(f"Rating range: {df_clean['rating_final'].min():.1f} to {df_clean['rating_final'].max():.1f}")

Final ratings: 970 records
Rating range: 1.0 to 5.0


## 3.5 Clean Review Counts


In [46]:
# Extract numbers from review count text
df_clean['review_count_clean'] = (
    df_clean['review_count_raw']
    .astype(str)
    .str.extract(r'(\d+)')[0]
    .astype("Int64")
)

print(f"Valid review counts: {df_clean['review_count_clean'].notna().sum()}")
print("Sample cleaned review counts:")
df_clean['review_count_clean'].dropna().head(5)

Valid review counts: 970
Sample cleaned review counts:


0     81
1     26
2    543
3     78
4    285
Name: review_count_clean, dtype: Int64

## 3.6 Clean Price Ranges

During the data quality check, we identified multiple currency formats in the price range data. Yelp, as an international platform, displays local currency symbols based on user location and business registration. Our dataset contains Euro (€), Dollar ($), and Pound (£) symbols, all representing legitimate price indicators from Yelp's Irish domain (Yelp.ie).

The presence of multiple currency formats reflects Yelp's global platform nature rather than data quality issues. We standardize all formats to Euro symbols for consistent analysis within the Irish market context. Regardless, the key is that all data genuinely comes from Yelp.ie, even if sometimes the data throws out a different currency.

We identified multiple currency symbols in the price data (€, $, £) from Yelp's platform. The cleaning process standardizes all currency formats to Euro symbols while preserving the price level information:

- Single symbol (`$`, `£`, `€`) → `€` (budget)
- Double symbols (`$$`, `££`, `€€`) → `€€` (moderate)  
- Triple symbols (`$$$`, `£££`, `€€€`) → `€€€` (premium)

This maintains the ordinal price level information while ensuring consistency for the Irish market analysis.

In [47]:
# Standardize all price formats to Euro symbols
def standardize_prices(price):
    if pd.isna(price):
        return np.nan
    price_str = str(price).strip()
    symbol_count = sum(1 for symbol in ['€', '$', '£'] if symbol in price_str)
    if symbol_count == 1:
        return '€'
    elif symbol_count == 2:
        return '€€'
    elif symbol_count >= 3:
        return '€€€'
    return np.nan

df_clean['price_clean'] = df_clean['price_range'].apply(standardize_prices)

print("Price ranges before cleaning:")
print(df_clean['price_range'].value_counts())
print("\nPrice ranges after cleaning:")
print(df_clean['price_clean'].value_counts())


# Convert to numerical values
price_mapping = {'€': 1, '€€': 2, '€€€': 3}
df_clean['price_encoded'] = df_clean['price_clean'].map(price_mapping)

# Fill missing prices with regional mode
def fill_prices(row):
    if pd.isna(row['price_encoded']):
        region_data = df_clean[df_clean['region'] == row['region']]
        mode_price = region_data['price_encoded'].mode()
        return mode_price.iloc[0] if len(mode_price) > 0 else 2
    return row['price_encoded']

df_clean['price_final'] = df_clean.apply(fill_prices, axis=1)

print(f"Missing prices after cleaning: {df_clean['price_final'].isna().sum()}")

Price ranges before cleaning:
price_range
€€      140
$$      113
€       108
$        95
£        31
££       26
€€€      17
$$$       5
€€€€      1
£££       1
Name: count, dtype: int64

Price ranges after cleaning:
price_clean
€    537
Name: count, dtype: int64
Missing prices after cleaning: 0


## 3.7 Clean Categories

In [52]:
# Create category count feature
df_clean['category_count'] = df_clean['categories'].str.split(',').str.len()

print("Category count summary:")
cat_stats = df_clean['category_count'].describe()
print(f"Count: {cat_stats['count']:.0f} businesses")
print(f"Average categories: {cat_stats['mean']:.1f}")
print(f"Standard deviation: {cat_stats['std']:.1f}")
print(f"Range: {cat_stats['min']:.0f} to {cat_stats['max']:.0f} categories")
print(f"25% have {cat_stats['25%']:.0f} or fewer categories")
print(f"50% have {cat_stats['50%']:.0f} or fewer categories") 
print(f"75% have {cat_stats['75%']:.0f} or fewer categories")

Category count summary:
Count: 970 businesses
Average categories: 3.7
Standard deviation: 1.3
Range: 2 to 10 categories
25% have 3 or fewer categories
50% have 4 or fewer categories
75% have 4 or fewer categories


 ## 3.8 Create Final Datasets

In [49]:
# Define the missing function for EDA dataset
def clean_ratings(rating):
    if pd.isna(rating):
        return np.nan
    rating_str = str(rating).lower()
    if any(word in rating_str for word in ['until', ':', 'closed']):
        return np.nan
    try:
        return float(rating)
    except:
        return np.nan

# EDA Dataset: Keep ALL records with cleaned values
df_eda = df_raw.copy()

print(f"Starting EDA dataset: {len(df_eda)} records")

# Apply cleaning but keep missing values for EDA
df_eda['rating_clean'] = df_eda['rating_raw'].apply(clean_ratings)
df_eda['review_count_clean'] = df_eda['review_count_raw'].astype(str).str.extract(r'(\d+)')[0].astype("Int64")
df_eda['price_clean'] = df_eda['price_range'].apply(standardize_prices)
df_eda['price_encoded'] = df_eda['price_clean'].map(price_mapping)
df_eda['category_count'] = df_eda['categories'].str.split(',').str.len()

print(f"Exploration dataset: {len(df_eda)} records")  # Should be 1,488

# Modeling Dataset: Use the already-cleaned df_clean (970 records)
df_model_encoded = pd.get_dummies(df_clean, columns=['region'], prefix='region')

print(f"Modeling dataset: {len(df_clean)} records")  # Should be 970

# Save both datasets
df_eda.to_csv("../data/dataProject_cleaned.csv", index=False)
df_model_encoded.to_csv("../data/dataProject_model.csv", index=False)

print("Datasets saved:")
print(f"dataProject_cleaned.csv: {len(df_eda)} records (all data for exploration)")
print(f"dataProject_model.csv: {len(df_clean)} records (complete cases for modeling)")

Starting EDA dataset: 1488 records
Exploration dataset: 1488 records
Modeling dataset: 970 records
Datasets saved:
dataProject_cleaned.csv: 1488 records (all data for exploration)
dataProject_model.csv: 970 records (complete cases for modeling)


## 3.9 Final Checks and Export

In [50]:
print("Final data quality check:")
print(f"Ratings: {df_clean['rating_final'].notna().sum()} valid")
print(f"Review counts: {df_clean['review_count_clean'].notna().sum()} valid") 
print(f"Prices: {df_clean['price_final'].notna().sum()} valid")

print(f"\nData retention: {len(df_clean)}/{len(df_raw)} records ({len(df_clean)/len(df_raw)*100:.1f}%)")

print("\nDataset summary:")
print(f"Raw data: {len(df_raw)} records")
print(f"Cleaned data: {len(df_clean)} records")
print(f"EDA data: {len(df_eda)} records")

Final data quality check:
Ratings: 970 valid
Review counts: 970 valid
Prices: 970 valid

Data retention: 970/1488 records (65.2%)

Dataset summary:
Raw data: 1488 records
Cleaned data: 970 records
EDA data: 1488 records


---
# **Data Cleaning Documentation**

This section documents the data cleaning process applied to the scraped Yelp dataset. The cleaning strategy prioritized data quality and integrity, using complete-case analysis to ensure reliable modeling while maintaining separate datasets for exploration and prediction.

---

## **3.1 Initial Data Assessment and Strategic Approach**

### **Raw Data Characteristics**
The original dataset contained **1,488 bakery listings** sourced from Yelp.ie across multiple Irish regions. Initial analysis revealed realistic data completeness patterns:

- **Rating Data**: 440 missing values (29.6% - new/unrated businesses)
- **Review Counts**: 516 missing (34.7% - businesses without customer reviews)
- **Price Ranges**: 940 missing (63.2% - listings without price indicators)
- **Categories**: All present but requiring standardization

### **Data Quality Issues Identified**
Manual inspection revealed specific data capture problems:
- **78 entries** with business hours incorrectly captured as ratings
- **272 entries** with mixed currency formats (€, $, £)
- **10 different price formats** requiring standardization

### **Cleaning Strategy**
We implemented a **complete-case analysis** approach for modeling data, prioritizing data integrity over quantity. This ensured all predictive modeling would use only genuine, complete business records.

---

## **3.2 Variable Classification Framework**

Before cleaning, we classified variables to guide appropriate treatment methods:

### **By Data Type**
- **Numerical**: rating_raw, review_count_raw
- **Categorical**: price_range, region, categories  
- **Text**: name, location, snippet
- **Metadata**: source

### **By Analytical Purpose**
- **Target Prediction**: rating_raw
- **Feature Predictors**: review_count_raw, price_range, region, categories
- **Context Information**: name, location, snippet, source

---

## **3.3 Comprehensive Data Cleaning Process**

### **3.3.1 Rating Data Cleaning**

**Challenge**: Rating data contained both data capture errors (business hours) and genuine missing values.

**Solution Implementation**:
1. **Remove Invalid Entries**: Identified and removed 78 entries where business hours were captured as ratings
2. **Type Conversion**: Converted valid ratings to numeric format using `pd.to_numeric()`
3. **Complete-Case Analysis**: Removed all remaining missing ratings to preserve modeling integrity

**Results**:
- Initial records: 1,488
- After cleaning: 970 valid ratings (65.2% retention)
- Rating range: 1.0 to 5.0 (valid distribution)

**Methodological Choice**: We chose complete-case analysis to avoid introducing artificial patterns and maintain the integrity of our predictive modeling target variable.

### **3.3.2 Review Count Processing**

**Challenge**: Review counts were stored as text strings (e.g., "125 reviews") requiring extraction.

**Solution Implementation**:
- Regular expression extraction (`r'(\d+)'`) captured numeric values
- `Int64` dtype preserved integer nature
- Maintained consistency with rating data filtering

**Results**: 970 valid review counts extracted and ready for analysis.

### **3.3.3 Price Range Standardization**

**Challenge**: Price indicators used multiple currency symbols and formats.

**Solution Implementation**:
1. **Standardize Formats**: Converted all currency symbols to consistent Euro format
2. **Ordinal Encoding**: Mapped to numerical values (€=1, €€=2, €€€=3)
3. **Regional Imputation**: Filled missing prices using regional modes

**Results**:
- All price formats standardized to Euro symbols
- 970 complete price records for modeling
- Ordinal relationships preserved for analysis

### **3.3.4 Category Processing**

**Solution Implementation**:
- Created `category_count` feature from categories field
- Enabled analysis of business diversification strategies

**Results**: Average of 3.7 categories per business, with range from 2 to 10 categories.

---

## **3.4 Strategic Dataset Creation**

### **EDA Dataset (`dataProject_cleaned.csv`)**
- **Purpose**: Comprehensive exploratory analysis
- **Records**: 1,488 (100% retention)
- **Characteristics**: All original records with cleaning applied, missing values retained
- **Use Case**: Understanding data patterns, distributions, and missing data relationships

### **Modeling Dataset (`dataProject_model.csv`)**
- **Purpose**: Predictive modeling with quality assurance
- **Records**: 970 (65.2% retention)
- **Characteristics**: Only complete cases for reliable predictions
- **Enhancements**: One-hot encoding for regions, all variables cleaned and validated

---

## **3.5 Data Quality Validation**

### **Final Data Quality**
- **Ratings**: 970 valid (1.0-5.0 range)
- **Review Counts**: 970 valid (extracted from text)
- **Prices**: 970 valid (standardized formats)
- **Categories**: All processed and counted

### **Data Retention**
- **Raw Data**: 1,488 records
- **Modeling Data**: 970 records (65.2% retention)
- **EDA Data**: 1,488 records (100% retention)

### **Methodological Strengths**
1. **Data Integrity**: All modeling uses genuine, complete cases
2. **Transparent Process**: Clear documentation of all cleaning decisions
3. **Dual Dataset Approach**: Separate data for exploration vs prediction
4. **Realistic Retention**: 65.2% reflects real-world business data patterns

---

## **3.6 Conclusion**

This data cleaning process successfully transformed raw web-scraped data into analysis-ready datasets while maintaining methodological rigor. The strategic decision to use complete-case analysis for modeling ensures predictive integrity, while the comprehensive EDA dataset enables thorough exploratory analysis.

The final datasets provide:
1. **Modeling Integrity**: 970 complete cases for reliable predictions
2. **Exploratory Power**: 1,488 records for comprehensive pattern analysis
3. **Data Quality**: All variables properly cleaned and validated
4. **Business Relevance**: Maintains real-world market characteristics

This foundation ensures robust exploratory analysis and predictive modeling for Irish bakery market insights.

---
# 4. Exploratory Data Analysis

In [51]:
# #Visualisation

# import matplotlib.pyplot as plt

# # Counting Bakeries by Region (YELP data only)
# df[df['source']=="Yelp"]['region'].value_counts().plot(kind='bar')
# plt.title("Number of Bakeries per Region (Yelp)")
# plt.xlabel("Region")
# plt.ylabel("Count")
# plt.show()

# # Ratings Distribution (also YELP data only)
# df['rating_raw'] = pd.to_numeric(df['rating_raw'], errors='coerce')

# df[df['source']=="Yelp"]['rating_raw'].plot(kind="hist", bins=10)
# plt.title("Distribution of Bakery Ratings")
# plt.xlabel("Rating")
# plt.ylabel("Frequency")
# plt.show()

---
# 5. Feature Engineering

---
# 6. Predictive Modelling

---
# 7. Findings and Conclusions

---

# Work Split per Member
Sofia Fedane
- Improved and documented the Data Mining Summary
- Performed all Data Cleaning tasks:
- variable typing
- variable purpose assignment
- missing value handling
- conversions (rating, reviews, price range, etc.)
- outlier treatment
- Completed all Univariate Analysis (numerical + categorical)
- Completed 3 Bivariate Analysis questions
- Performed all baseline regression modelling, including:
- feature engineering
- one-hot encoding
- train/test split
- Linear Regression model
- coefficient interpretation
- Wrote the Findings & Conclusions section


Iker Arza
- Wrote the Business Understanding section
- Completed the remaining 3 Bivariate Analysis questions
- Performed the full Multivariate Analysis:
- correlation matrix
- region × price_range heatmap
- top 10% high-rating analysis
- Implemented the advanced regression models:
- Random Forest Regressor
- Gradient Boosting Regressor
- Produced the model comparison table
- Selected and justified the final recommended model
- Wrote docucentation for advanced modelling and interpretations
- Shared Responsibilities
- Wrote the Modelling Introduction
- justified regression choice
- defined the response variable
- listed predictor variables
- stated modelling limitations